# Marketing Mix Modeling
## Marketing Mix Modeling - a method that helps to quantify the impact of marketing investments on some target-variable.  Unlike the usual regression, which can only analyze dependencies between two variables, MMM focuses on how several variables affect the result.

## The main advantage of Marketing Mix Modeling is that the model takes into account two important advertising effects:
### saturation - cannot achieve multiple increase of variable target at huge cost
### ad-stock - when money is spent on advertising today, but tomorrow (and even after the end of the campaign) the effect remains in people’s memories

## Google’s LightweightMMM library was used for the Marketing Mix Modeling method. This library is designed as a faster and easier solution that allows to get detailed results for analysis. In addition to the standard approach, LightweightMMM offers a hierarchical approach. If there is data at the state or regional level, this hierarchical approach based on geography can give more accurate results.

In [ ]:
## Sources:
# https://colab.research.google.com/drive/1JjFFXVsni3KQ73bSjiutZ9tLjQlcI652?pli=1#scrollTo=qtt9L4aPL-cs
# https://towardsdatascience.com/media-mix-modeling-how-to-measure-the-effectiveness-of-advertising-with-python-lightweightmmm-b6d7de110ae6/
# https://medium.com/@mail2rajivgopinath/setting-priors-in-bayesian-marketing-mix-modeling-mmm-9f1b7469d96c
# https://medium.com/@thomascgeorgiou/media-mix-modelling-measuring-advertising-effectiveness-using-googles-lightweightmmm-python-80cb4974b6ce
# https://medium.com/@camilojaure/exploring-the-feasibility-of-lightweight-mmm-for-holistic-marketing-attribution-d587bcff0429
# https://towardsdatascience.com/mastering-marketing-mix-modelling-in-python-7bbfe31360f9/
# https://www.kaggle.com/code/jordangronkowski/lightweightmmm-for-marketing-mix-modeling
# https://github.com/google/lightweight_mmm

In [ ]:
%matplotlib inline

import pandas as pd
import numpy as np
from io import StringIO
import sys
import statsmodels.api as sm
from os import listdir
from os.path import isfile, join
import matplotlib.pyplot as plt
import math
import warnings
warnings.filterwarnings('ignore')
import seaborn as sb

from statsmodels.stats.outliers_influence import variance_inflation_factor

In [ ]:
# Import jax.numpy and any other library we might need.
import jax.numpy as jnp
import numpyro

# Import the relevant modules of the library
from lightweight_mmm import lightweight_mmm
from lightweight_mmm import optimize_media
from lightweight_mmm import plot
from lightweight_mmm import preprocessing
from lightweight_mmm import utils
from lightweight_mmm import models

from sklearn.metrics import mean_absolute_percentage_error
from sklearn.preprocessing import StandardScaler

from scipy.stats import chi2

# Explarotary Data Analysis

In [ ]:
df = pd.read_excel(f'Data_for_MMM.xlsx')
df.describe()

In [ ]:
# Interpolation for macroeconomic factors
df["macroeconomic_factor_1"] = df["macroeconomic_factor_1_inter"].interpolate()
df["macroeconomic_factor_2"] = df["macroeconomic_factor_2_inter"].interpolate()

In [ ]:
df.describe()

## Correlation

In [ ]:
corr = df.drop('sales', axis=1).corr()
corr

In [ ]:
# Plotting correlation heatmap
dataplot = sb.heatmap(df.corr(numeric_only=True), cmap="YlGnBu", annot=True)

# Displaying heatmap
plt.show()

## Multicollinearity

In [ ]:
variables = ['media_channel_1', 'media_channel_2', 'media_channel_3', 'media_channel_4', 'media_channel_5', 
             'macroeconomic_factor_1', 'macroeconomic_factor_2', 
             'macroeconomic_factor_1_inter', 'macroeconomic_factor_2_inter']

vif_data = pd.DataFrame()
X = df[variables]
vif_data['feature'] = X.columns
vif_data['VIF'] = [variance_inflation_factor(X.values, i) for i in range(len(X.columns))]
print(vif_data)

## Plot to vizualize the amount of media channel expenses

In [ ]:
plt.figure(figsize=(12, 6))

# Stacked Area Plot
df[['media_channel_1', 'media_channel_2', 'media_channel_3', 'media_channel_4', 'media_channel_5']].plot.area(alpha=0.8, figsize=(12, 6))

# Quality Data Check

## Test, train data and scaling

In [ ]:
mdsp_cols = ['media_channel_1', 'media_channel_2', 'media_channel_3', 'media_channel_4', 'media_channel_5']
test_data_period_size = 4
costs_multiply = 0.15
df['index1'] = df.index
control_vars = ['macroeconomic_factor_1', 'macroeconomic_factor_2']
sales_cols =['sales']

df_main = df[['index1']+sales_cols+mdsp_cols+control_vars]

In [ ]:
SEED = 105
data_size = len(df_main)

n_media_channels = len(mdsp_cols)
n_extra_features = len(control_vars)
media_data = df_main[mdsp_cols].to_numpy()
extra_features = df_main[control_vars].to_numpy()
target = df_main['sales'].to_numpy()
costs = df_main[mdsp_cols].sum().to_numpy()

# Split and scale data.
test_data_period_size = test_data_period_size
split_point = data_size - test_data_period_size
# Media data
media_data_train = media_data[:split_point, ...]
media_data_test = media_data[split_point:, ...]
# Extra features
extra_features_train = extra_features[:split_point, ...]
extra_features_test = extra_features[split_point:, ...]
# Target
target_train = target[:split_point]
target_test = target[split_point:]

def nonzero_mean(arr):
    return jnp.nanmean(jnp.where(arr != 0, arr, jnp.nan))

media_scaler = preprocessing.CustomScaler(divide_operation=nonzero_mean)

extra_features_scaler = preprocessing.CustomScaler(divide_operation=nonzero_mean)

target_scaler = preprocessing.CustomScaler(divide_operation=jnp.mean)
# multiply_by = 0.15 for normalizing data
if costs_multiply == 0:
    cost_scaler = preprocessing.CustomScaler(divide_operation=jnp.mean)
else:
    cost_scaler = preprocessing.CustomScaler(divide_operation=jnp.mean, multiply_by = costs_multiply)

media_data_train = media_scaler.fit_transform(media_data_train)
extra_features_train = extra_features_scaler.fit_transform(extra_features_train)
target_train = target_scaler.fit_transform(target_train)
costs = cost_scaler.fit_transform(costs)

In [ ]:
correlations, variances, spend_fractions, variance_inflation_factors = preprocessing.check_data_quality(
    media_data=media_scaler.transform(media_data),
    target_data=target_scaler.transform(target),
    cost_data=costs,
    extra_features_data=extra_features_scaler.transform(extra_features))

## Correlation Matrix

In [ ]:
# The below cell shows the correlation matrix between all the features, and between each feature and the target. 
# Very positive or very negative correlations (with an absolute value above, say, 0.7 or so) 
# should be treated with caution. In this case you might consider dropping or 
# merging highly correlated features.
correlations[0].style.background_gradient(cmap='RdBu', vmin=-1, vmax=1).format(precision=3)

## Variance

In [ ]:
# The below cell shows the variance of each feature over time. 
# Variances which are lower than the specified low_variance_threshold or higher 
# than the specified high_variance_threshold are marked in red. 
# Make sure you are passing the scaled versions of your media_data and extra_features_data 
# to the data quality checker for these variances before running this check!

def highlight_variances(x: float, 
                        low_variance_threshold: float=1.0e-3, 
                        high_variance_threshold: float=3.0) -> str:

    if x < low_variance_threshold or x > high_variance_threshold:
      weight = 'bold'
      color = 'red'
    else:
      weight = 'normal'
      color = 'black'
    style = f'font-weight: {weight}; color: {color}'
    return style

variances.style.format(precision=4).applymap(highlight_variances)

## Low spend fractions

In [ ]:
def highlight_low_spend_fractions(x: float,
                                  low_spend_threshold: float=0.01) -> str:
    if x < low_spend_threshold:
      weight = 'bold'
      color = 'red'
    else:
      weight = 'normal'
      color = 'black'
    style = f'font-weight: {weight}; color: {color}'
    return style

spend_fractions.style.format(precision=4).applymap(highlight_low_spend_fractions)

## Multicollinearity

In [ ]:
# While checking the correlation matrix in step 1 is usually sufficient 
# for detecting obvious multicollinearity in a dataset, the variance inflation factor is 
# technically the best metric for identifying multicollinear features. 
# Here we list the variance inflation factors for all features. If the number is too high 
# (we use a threshold here of 7, but feel free to adjust to your use case) 
# you might consider dropping or merging features with high variance inflation factors.

def highlight_high_vif_values(x: float,
                              high_vif_threshold: float=7.0) -> str:
    if x > high_vif_threshold:
      weight = 'bold'
      color = 'red'
    else:
      weight = 'normal'
      color = 'black'
    style = f'font-weight: {weight}; color: {color}'
    return style

variance_inflation_factors.style.format(precision=4).applymap(highlight_high_vif_values)

# Marketing Mix Modeling

In [ ]:
SEED = 105
data_size = len(df_main)

n_media_channels = len(mdsp_cols)
n_extra_features = len(control_vars)
media_data = df_main[mdsp_cols].to_numpy()
extra_features = df_main[control_vars].to_numpy()
target = df_main['sales'].to_numpy()
costs = df_main[mdsp_cols].sum().to_numpy()

# Split and scale data.
test_data_period_size = test_data_period_size
split_point = data_size - test_data_period_size
# Media data
media_data_train = media_data[:split_point, ...]
media_data_test = media_data[split_point:, ...]
# Extra features
extra_features_train = extra_features[:split_point, ...]
extra_features_test = extra_features[split_point:, ...]
# Target
target_train = target[:split_point]
target_test = target[split_point:]

def nonzero_mean(arr):
    return jnp.nanmean(jnp.where(arr != 0, arr, jnp.nan))


from sklearn.preprocessing import StandardScaler

media_scaler = preprocessing.CustomScaler(divide_operation=nonzero_mean)

#extra_features_scaler = preprocessing.CustomScaler(divide_operation=jnp.mean)
#extra_features_scaler = StandardScaler()

extra_features_scaler = preprocessing.CustomScaler(divide_operation=nonzero_mean)

target_scaler = preprocessing.CustomScaler(divide_operation=jnp.mean)
# multiply_by = 0.15 for normalizing data
if costs_multiply == 0:
    cost_scaler = preprocessing.CustomScaler(divide_operation=jnp.mean)
else:
    cost_scaler = preprocessing.CustomScaler(divide_operation=jnp.mean, multiply_by = costs_multiply)
#cost_scaler = preprocessing.CustomScaler(divide_operation=jnp.mean)

media_data_train = media_scaler.fit_transform(media_data_train)
extra_features_train = extra_features_scaler.fit_transform(extra_features_train)
target_train = target_scaler.fit_transform(target_train)
costs = cost_scaler.fit_transform(costs)

In [ ]:
# function to calculate Weighted Average Percentage Error
def wape(y_true, y_pred):
    error = np.abs(y_true - y_pred)
    return np.sum(error) / np.sum(y_true)

# function to get data from marketing mix modeling summary
def summary_df(s):
    # Custom processing 
    # Due to noise in output I couldn't get reading directly from string buffer into pandas read csv to function)
    # Row separators are '\n' and Col separators are whitespace '\\s+'
    rows = []
    cols = re.split('\\s+', s.split('\n')[1])[1:]
    print(s.split('\n')[-2:][0]) # Divergences
    for row in [ln for i, ln in enumerate(s.split('\n')) if i > 1]:
        split_row = re.split('\\s+', row)[1:]
        # For 3D parameters (gamma_seasonality)
        if len(split_row) == 7:
            rows += [['', *split_row]]
        elif len(split_row) == 8:
            rows += [split_row]

    return pd.DataFrame(rows, columns=['Param', *cols] )

In [ ]:
# function to choose the best model parameters on a test dataset
# to identify model quality MAPE metric (also WAPE metric) is calculated and R_hat is checked to be less than 1.1

def mmm_test_choose(df):
    adstock_models = ["adstock", "hill_adstock", "carryover"]
    degrees_season = [1,2,3]
    
    best_mape = float('inf')
    best_model = None
    for model_name in adstock_models:
      for degrees in degrees_season:
        mmm = lightweight_mmm.LightweightMMM(model_name=model_name)
        mmm.fit(media=media_data_train,
                media_prior=costs,
                target=target_train,
                extra_features=extra_features_train,
                number_warmup=1000,
                number_samples=1000,
               # number_chains=1,
                degrees_seasonality=degrees,
               # weekday_seasonality=True,
                seasonality_frequency=12,
                # custom_priors = {'lag_weight': {'concentration1': 4., 'concentration0': 1.}},
                seed=105)
        prediction = mmm.predict(
        media=media_scaler.transform(media_data_test),
        extra_features=extra_features_scaler.transform(extra_features_test),
        target_scaler=target_scaler)
        p = prediction.mean(axis=0)
        mape = mean_absolute_percentage_error(target_test, p)
    
        f = io.StringIO()
        with redirect_stdout(f):
            mmm.print_summary()
        df_mmm = summary_df(f.getvalue())
        if len(df_mmm.loc[df_mmm.r_hat.astype(float) >= 1.1]) > 0:
            r_hat = "R_HAT >= 1.1"
        else:
            r_hat = "R_HAT ok"
        print(f"model_name={model_name} degrees={degrees} MAPE={mape} samples={p[:3]} {r_hat}")
    
        print(f"WAPE_test:{wape(target_test, p)*100} {r_hat}\n")
          
        if mape < best_mape and r_hat == "R_HAT ok":
          best_mape = mape
          best_model = f"model_name={model_name} degrees={degress_seasonality} mape={mape}"
          
    print("Best Performing Model:", best_model, " \n")


In [ ]:
# function takes all needed data for mmm model (marketing mix model) and all needed technical parameters
# as a result, a data frame with percentages of media contribution for each media channel towards the target value is provided for each month

def function_mmm(mdsp_cols, control_vars, test_data_period_size, model_name, degrees_seasonality, costs_multiply):
    df['index1'] = df.index
    sales_cols =['sales']
    
    df_main = df[['index1']+sales_cols+mdsp_cols+control_vars]

    SEED = 105
    data_size = len(df_main)
    
    n_media_channels = len(mdsp_cols)
    n_extra_features = len(control_vars)
    media_data = df_main[mdsp_cols].to_numpy()
    extra_features = df_main[control_vars].to_numpy()
    target = df_main['sales'].to_numpy()
    costs = df_main[mdsp_cols].sum().to_numpy()
    
    # Split and scale data.
    # test_data_period_size = test_data_period_size
    split_point = data_size - test_data_period_size
    # Media data
    media_data_train = media_data[:split_point, ...]
    media_data_test = media_data[split_point:, ...]
    # Extra features
    extra_features_train = extra_features[:split_point, ...]
    extra_features_test = extra_features[split_point:, ...]
    # Target
    target_train = target[:split_point]
    target_test = target[split_point:]
    
    def nonzero_mean(arr):
        return jnp.nanmean(jnp.where(arr != 0, arr, jnp.nan))
    
    media_scaler = preprocessing.CustomScaler(divide_operation=nonzero_mean)
    extra_features_scaler = preprocessing.CustomScaler(divide_operation=nonzero_mean)
    
    target_scaler = preprocessing.CustomScaler(divide_operation=jnp.mean)
    # multiply_by = 0.15 for normalizing data
    if costs_multiply == 0:
        cost_scaler = preprocessing.CustomScaler(divide_operation=jnp.mean)
    else:
        cost_scaler = preprocessing.CustomScaler(divide_operation=jnp.mean, multiply_by = costs_multiply)
    #cost_scaler = preprocessing.CustomScaler(divide_operation=jnp.mean)
    
    media_data_train = media_scaler.fit_transform(media_data_train)
    extra_features_train = extra_features_scaler.fit_transform(extra_features_train)
    target_train = target_scaler.fit_transform(target_train)
    costs = cost_scaler.fit_transform(costs)

    print(f"costs_multiply: {costs_multiply}\n\ncontrol_vars = {control_vars}\n\nmdsp_cols = {mdsp_cols}\n\nmodel_name: {model_name}, deg_seasonality: {degrees_seasonality}")
    
    number_warmup=1000
    number_samples=1000
    mmm = lightweight_mmm.LightweightMMM(model_name=model_name)
    
    mmm.fit(media=media_data_train, media_prior=costs, target=target_train,
             extra_features=extra_features_train,
            degrees_seasonality=degrees_seasonality,
            seasonality_frequency=12,
            number_warmup=number_warmup, number_samples=number_samples, media_names = mdsp_cols,
            seed=SEED)

    '''Once training is finished, you can check the summary of your trace: 
    The important point here is to check whether r hat values for all parameters are less than 1.1. 
    This is a checkpoint when you run Bayesian modeling.'''
    # Capture Output to stdout by print summary
    f = io.StringIO()
    with redirect_stdout(f):
        mmm.print_summary()
    df_mmm = summary_df(f.getvalue())
    
    if len(df_mmm.loc[df_mmm.r_hat.astype(float) >= 1.1]) > 0:
        print("R_HAT >= 1.1")
    else:
        print("R_HAT ok")

    plot.plot_model_fit(mmm, target_scaler=target_scaler).savefig(f'prediction_{added_dt}.png')

    media_contribution, roi_hat = mmm.get_posterior_metrics(target_scaler=target_scaler, cost_scaler=cost_scaler)
    plot.plot_media_baseline_contribution_area_plot(media_mix_model=mmm,
                                                    target_scaler=target_scaler,
                                                    fig_size=(20,10),
                                                    channel_names = mdsp_cols
                                                    ).savefig(f'media_contr_{added_dt}.png')

    #WAPE
    true_train = target_scaler.inverse_transform(target_train)
    posterior_pred = mmm.trace["mu"]
    posterior_pred = target_scaler.inverse_transform(posterior_pred)
    p = posterior_pred.mean(axis = 0)
    
    print(f"WAPE_train:{wape(true_train, p)*100}\n")

    
    # Getting percentages of contribution
    media_contributionss = jnp.einsum("stc, sc->stc",
                                  mmm.trace["media_transformed"],
                                  mmm.trace["coef_media"])

    mean_contributions = np.mean(media_contributionss, axis=0) # Shape: (51, 6)

    total_contributions_per_time = np.sum(mean_contributions, axis=1)  # Shape: (51,)
    
    # Step 3: Calculate percentage contribution for each channel at each time point
    percentage_contributions = (mean_contributions.T / total_contributions_per_time).T * 100  # Shape: (51, 6)
    
    # Step 4: Create a DataFrame for better visualization
    df['month'] = df['index1']
    time_points_range = df['index1']
    channel_names = mdsp_cols
    percentage_contributions_df = pd.DataFrame(percentage_contributions, columns=channel_names, index=time_points_range)
    mean_contributions_df = pd.DataFrame(mean_contributions, columns=channel_names, index=time_points_range)
    
    # composing the final data frame
    percentages = df[['yyyymm', 'index1']].merge(percentage_contributions_df.reset_index(), on = 'index1', how = 'outer')
    percentages = percentages.drop(columns = 'index1', axis = 1)
    
    return percentages

In [ ]:
# Choosing the right parametrs for the model
mmm_test_choose(df)

In [ ]:
# Getting the monthly data frame with percentages of media contribution for each media channel towards the target value
df_mmm = mmm(i, df)

# Checking whether interpolated macroeconomic factors and adding the holidays data improve the model 

In [ ]:
# function takes all needed data for mmm model (marketing mix modeling) and all needed technical parameters
# as a result, data for considering whether interpolation and holidays improve the model

def function_mmm(mdsp_cols, control_vars, test_data_period_size,  model_name, degrees_seasonality, costs_multiply):
    df['index1'] = df.index
    sales_cols =['sales']
    
    df_main = df[['index1']+sales_cols+mdsp_cols 
    +control_vars
    ]

    SEED = 105
    data_size = len(df_main)
    
    n_media_channels = len(mdsp_cols)
    n_extra_features = len(control_vars)
    media_data = df_main[mdsp_cols].to_numpy()
    extra_features = df_main[control_vars].to_numpy()
    target = df_main['sales'].to_numpy()
    costs = df_main[mdsp_cols].sum().to_numpy()
    
    # Split and scale data.
    # test_data_period_size = test_data_period_size
    split_point = data_size - test_data_period_size
    # Media data
    media_data_train = media_data[:split_point, ...]
    media_data_test = media_data[split_point:, ...]
    # Extra features
    extra_features_train = extra_features[:split_point, ...]
    extra_features_test = extra_features[split_point:, ...]
    # Target
    target_train = target[:split_point]
    target_test = target[split_point:]
    
    def nonzero_mean(arr):
        return jnp.nanmean(jnp.where(arr != 0, arr, jnp.nan))
    
    
    from sklearn.preprocessing import StandardScaler
    
    media_scaler = preprocessing.CustomScaler(divide_operation=nonzero_mean)
    
    #extra_features_scaler = preprocessing.CustomScaler(divide_operation=jnp.mean)
    #extra_features_scaler = StandardScaler()
    
    extra_features_scaler = preprocessing.CustomScaler(divide_operation=nonzero_mean)
    
    target_scaler = preprocessing.CustomScaler(divide_operation=jnp.mean)
    # multiply_by = 0.15 for normalizing data
    if costs_multiply == 0:
        cost_scaler = preprocessing.CustomScaler(divide_operation=jnp.mean)
    else:
        cost_scaler = preprocessing.CustomScaler(divide_operation=jnp.mean, multiply_by = costs_multiply)
    #cost_scaler = preprocessing.CustomScaler(divide_operation=jnp.mean)
    
    media_data_train = media_scaler.fit_transform(media_data_train)
    extra_features_train = extra_features_scaler.fit_transform(extra_features_train)
    target_train = target_scaler.fit_transform(target_train)
    costs = cost_scaler.fit_transform(costs)

    number_warmup=1000
    number_samples=1000
    mmm = lightweight_mmm.LightweightMMM(model_name=model_name)
    
    mmm.fit(media=media_data_train, media_prior=costs, target=target_train,
             extra_features=extra_features_train,
            degrees_seasonality=degrees_seasonality,
            seasonality_frequency=12,
            number_warmup=number_warmup, number_samples=number_samples, media_names = mdsp_cols,
            seed=SEED)

    prediction = mmm.predict(
    media=media_scaler.transform(media_data_test),
    extra_features=extra_features_scaler.transform(extra_features_test),
    target_scaler=target_scaler)

    residuals = target_test - prediction
    sigma_sq = np.var(residuals)
    log_likelihood = -0.5 * np.sum((residuals ** 2) / sigma_sq + np.log(2 * np.pi * sigma_sq))

    p = prediction.mean(axis=0)
    mape = mean_absolute_percentage_error(target_test, p)
    
    return extra_features_train, log_likelihood, mape, mmm

In [ ]:
# Putting the needed data for function_mmm() 

mdsp_cols = ['media_channel_1', 'media_channel_2', 'media_channel_3', 'media_channel_4', 'media_channel_5']
test_data_period_size = 4
degrees_seasonality = 2
costs_multiply = 0.15

# model_name and degrees_seasonality are chosen by mmm_test_choose(df) function
model_name = 'hill_adstock' 
degrees_seasonality = 2

In [ ]:
extra_features_train_baseline, log_likelihood_baseline, mape_baseline, mmm_baseline = function_mmm(mdsp_cols = mdsp_cols, 
                            test_data_period_size = test_data_period_size,
                            control_vars = ['macroeconomic_factor_1', 'macroeconomic_factor_2'],
                            model_name = model_name,
                            degrees_seasonality = degrees_seasonality,
                            costs_multiply = costs_multiply)

extra_features_train_control, log_likelihood_control, mape_control, mmm_control = function_mmm(mdsp_cols = mdsp_cols, 
                            test_data_period_size = test_data_period_size,
                            control_vars = ['macroeconomic_factor_1', 'macroeconomic_factor_2', 'holidays'],
                            model_name = model_name,
                            degrees_seasonality = degrees_seasonality,
                            costs_multiply = costs_multiply)

extra_features_train_baseline_inter, log_likelihood_baseline_inter, mape_baseline_inter, mmm_baseline_inter = function_mmm(mdsp_cols = mdsp_cols, 
                            test_data_period_size = test_data_period_size,
                            control_vars = ['macroeconomic_factor_1_inter', 'macroeconomic_factor_2_inter'],
                            model_name = model_name,
                            degrees_seasonality = degrees_seasonality,
                            costs_multiply = costs_multiply)

extra_features_train_control_inter, log_likelihood_control_inter, mape_control_inter, mmm_control_inter = function_mmm(mdsp_cols = mdsp_cols, 
                            test_data_period_size = test_data_period_size,
                            control_vars = ['macroeconomic_factor_1_inter', 'macroeconomic_factor_2_inter', 'holidays'],
                            model_name = model_name,
                            degrees_seasonality = degrees_seasonality,
                            costs_multiply = costs_multiply)


In [ ]:
# The results whether control variable (holidays) and interpolation significantly improves the model are shown

# Compute LRT statistic
lrt_stat = -2 * (log_likelihood_baseline - log_likelihood_control)
lrt_stat_inter = -2 * (log_likelihood_baseline_inter - log_likelihood_control_inter)

# Degrees of freedom (difference in model parameters)
df_diff = extra_features_train_control.shape[1] - extra_features_train_baseline.shape[1]
df_diff_inter = extra_features_train_control_inter.shape[1] - extra_features_train_baseline_inter.shape[1]

# Compute p-value
p_value = 1 - chi2.cdf(lrt_stat, df=df_diff)
p_value_inter = 1 - chi2.cdf(lrt_stat, df=df_diff_inter)

print('DATA WITHOUT INTERPOLATION')
# Print results
print("Baseline Log-Likelihood:", log_likelihood_baseline)
print("Control Model Log-Likelihood:", log_likelihood_control)
print("LRT Statistic:", lrt_stat)
print("p-value:", p_value)

print('INTERPOLATED DATA')
# Print results
print("Baseline Log-Likelihood_inter:", log_likelihood_baseline_inter)
print("Control Model Log-Likelihood_inter:", log_likelihood_control_inter)
print("LRT Statistic Interpolation:", lrt_stat_inter)
print("p-value interpolation:", p_value_inter)

print('DATA WITHOUT INTERPOLATION')
# control variable = holidays
if p_value < 0.05:
    print("✅ The control variable significantly improves the model!")
else:
    print("❌ The control variable does NOT significantly improve the model.")

print('INTERPOLATED DATA')
# control variable = holidays
if p_value_inter < 0.05:
    print("✅ The control variable significantly improves the model!")
else:
    print("❌ The control variable does NOT significantly improve the model.")

print('DATA WITHOUT INTERPOLATION')
print("Baseline MAPE:", mape_baseline)
print("With Control MAPE:", mape_control)

print('INTERPOLATED DATA')
print("Baseline MAPE inter:", mape_baseline_inter)
print("With Control MAPE inter:", mape_control_inter)

print("WITHOUT HOLIDAYS")
if mape_baseline_inter < mape_baseline:
    print("✅ The interpolation significantly improves the model!")
else:
    print("❌ The interpolation does NOT significantly improve the model.")

print("WITH HOLIDAYS")
if mape_control_inter < mape_control:
    print("✅ The interpolation significantly improves the model!")
else:
    print("❌ The interpolation does NOT significantly improve the model.")

# Getting more specific results

In [ ]:
number_warmup=1000
number_samples=1000
mmm = lightweight_mmm.LightweightMMM(model_name=model_name)
mmm.fit( media=media_data_train, media_prior=costs, target=target_train,
         extra_features=extra_features_train,
        degrees_seasonality=degrees_seasonality,
        seasonality_frequency=12,
        number_warmup=number_warmup, number_samples=number_samples, media_names = mdsp_cols, seed=SEED)

In [ ]:
'''Once training is finished, you can check the summary of your trace: 
The important point here is to check whether r hat values for all parameters are less than 1.1. 
This is a checkpoint when you run Bayesian modeling.'''
mmm.print_summary()

In [ ]:
# Checking R_hat
f = io.StringIO()
with redirect_stdout(f):
    mmm.print_summary()
df_mmm = summary_df(f.getvalue())
if len(df_mmm.loc[df_mmm.r_hat.astype(float) >= 1.1]) > 0:
    r_hat = "R_HAT >= 1.1"
else:
    r_hat = "R_HAT ok"

In [ ]:
# Obtaining a plot of the train dataset
plot.plot_model_fit(mmm, target_scaler=target_scaler)

In [ ]:
# Obtaining a plot of the media contribution 

media_contribution, roi_hat = mmm.get_posterior_metrics(target_scaler=target_scaler, cost_scaler=cost_scaler)
plot.plot_media_baseline_contribution_area_plot(media_mix_model=mmm,
                                                target_scaler=target_scaler,
                                                fig_size=(20,10),
                                                channel_names = mdsp_cols
                                                )

In [ ]:
# Obtaining a plot of the test dataset
# We have to scale the test media data if we have not done so before

new_predictions = mmm.predict(media=media_scaler.transform(media_data_test),
                              extra_features=extra_features_scaler.transform(extra_features_test),
                              seed=SEED)
print(new_predictions.shape)

plot.plot_out_of_sample_model_fit(out_of_sample_predictions=new_predictions,
                                 out_of_sample_target=target_scaler.transform(target[split_point:]))

In [ ]:
# function to calculate Weighted Average Percentage Error
def wape(y_true, y_pred):
    error = np.abs(y_true - y_pred)
    return np.sum(error) / np.sum(y_true)

# WAPER test and WAPE train
prediction = mmm.predict(
    media=media_scaler.transform(media_data_test),
    extra_features=extra_features_scaler.transform(extra_features_test),
    target_scaler=target_scaler,
    seed=SEED)
p = prediction.mean(axis = 0)

print(f"WAPE_test:{wape(target_test, p)*100}")

true_train = target_scaler.inverse_transform(target_train)

posterior_pred = mmm.trace["mu"]
posterior_pred = target_scaler.inverse_transform(posterior_pred)
p = posterior_pred.mean(axis = 0)

print(f"WAPE_train:{wape(true_train, p)*100}")

In [ ]:
# Obtaining coefficients for each media channel
mdsp_cols = mdsp_cols
plot.plot_media_channel_posteriors(media_mix_model=mmm, channel_names = mdsp_cols)

In [ ]:
# Getting percentages of contribution

media_contributionss = jnp.einsum("stc, sc->stc",
                              mmm.trace["media_transformed"],
                              mmm.trace["coef_media"])

mean_contributions = np.mean(media_contributionss, axis=0) # Shape: (51, 6)

total_contributions_per_time = np.sum(mean_contributions, axis=1)  # Shape: (51,)

# Step 3: Calculate percentage contribution for each channel at each time point
percentage_contributions = (mean_contributions.T / total_contributions_per_time).T * 100  # Shape: (51, 6)

# Step 4: Create a DataFrame for better visualization
df['month'] = df['index1']
time_points_range = df['index1']
channel_names = mdsp_cols
percentage_contributions_df = pd.DataFrame(percentage_contributions, columns=channel_names, index=time_points_range)
mean_contributions_df = pd.DataFrame(mean_contributions, columns=channel_names, index=time_points_range)

In [ ]:
# data frame with media contribution in %
excel_data = df.merge(percentage_contributions_df.reset_index(), on = 'index1', how = 'outer', suffixes = (f'', f'_contr_%'))
excel_data = excel_data.drop(columns = ['index1', 'month'], axis = 1)

In [ ]:
# Media Contribution for the last month

# Select the single row (as Series)
row = percentage_contributions_df.iloc[-1:].iloc[0]

# Create a pie chart
plt.figure(figsize=(7, 7))
row.plot.pie(
    autopct="%1.1f%%",  # Display percentage
    startangle=80,      # Start at 90 degrees for better orientation
    legend=True,        # Add legend
    ylabel="",          # Remove default ylabel
    title="Media Channel Contributions", fontsize=12
)

# Show the plot
plt.tight_layout()
plt.show()

In [ ]:
# Obtaining the plot of nedia channel effectiveness
# Shows how much sales a dollar in the media channel brings
plot.plot_bars_media_metrics(metric=media_contributionss, metric_name="Aquisition effectiveness", channel_names=mdsp_cols)

In [ ]:
# Curves for each media channel
plot.plot_response_curves(media_mix_model=mmm, target_scaler=target_scaler, seed=SEED)